In [1]:
"""
This part of code is the Deep Q Network (DQN) brain.
view the tensorboard picture about this DQN structure on: https://morvanzhou.github.io/tutorials/machine-learning/reinforcement-learning/4-3-DQN3/#modification
View more on my tutorial page: https://morvanzhou.github.io/tutorials/
Using:
Tensorflow: r1.2
"""

import numpy as np
import tensorflow as tf

np.random.seed(1)
tf.set_random_seed(1)


# Deep Q Network off-policy
class DeepQNetwork:
    def __init__(
            self,
            n_actions,
            n_features,
            learning_rate=0.01,
            reward_decay=0.9,
            e_greedy=0.9,
            replace_target_iter=300,
            memory_size=500,
            batch_size=32,
            e_greedy_increment=None,
            output_graph=False,
    ):
        self.n_actions = n_actions
        self.n_features = n_features
        self.lr = learning_rate
        self.gamma = reward_decay
        self.epsilon_max = e_greedy
        self.replace_target_iter = replace_target_iter
        self.memory_size = memory_size
        self.batch_size = batch_size
        self.epsilon_increment = e_greedy_increment
        self.epsilon = 0 if e_greedy_increment is not None else self.epsilon_max

        # total learning step
        self.learn_step_counter = 0

        # initialize zero memory [s, a, r, s_]
        self.memory = np.zeros((self.memory_size, n_features * 2 + 2))

        # consist of [target_net, evaluate_net]
        self._build_net()

        t_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        e_params = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='eval_net')

        with tf.variable_scope('hard_replacement'):
            self.target_replace_op = [tf.assign(t, e) for t, e in zip(t_params, e_params)]

        self.sess = tf.Session()

        if output_graph:
            # $ tensorboard --logdir=logs
            tf.summary.FileWriter("logs/", self.sess.graph)

        self.sess.run(tf.global_variables_initializer())
        self.cost_his = []

    def _build_net(self):
        # ------------------ all inputs ------------------------
        self.s = tf.placeholder(tf.float32, [None, self.n_features], name='s')  # input State
        self.s_ = tf.placeholder(tf.float32, [None, self.n_features], name='s_')  # input Next State
        self.r = tf.placeholder(tf.float32, [None, ], name='r')  # input Reward
        self.a = tf.placeholder(tf.int32, [None, ], name='a')  # input Action

        w_initializer, b_initializer = tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)

        # ------------------ build evaluate_net ------------------
        with tf.variable_scope('eval_net'):
            e1 = tf.layers.dense(self.s, 20, tf.nn.relu, kernel_initializer=w_initializer,
                                 bias_initializer=b_initializer, name='e1')
            self.q_eval = tf.layers.dense(e1, self.n_actions, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='q')

        # ------------------ build target_net ------------------
        with tf.variable_scope('target_net'):
            t1 = tf.layers.dense(self.s_, 20, tf.nn.relu, kernel_initializer=w_initializer,
                                 bias_initializer=b_initializer, name='t1')
            self.q_next = tf.layers.dense(t1, self.n_actions, kernel_initializer=w_initializer,
                                          bias_initializer=b_initializer, name='t2')

        with tf.variable_scope('q_target'):
            q_target = self.r + self.gamma * tf.reduce_max(self.q_next, axis=1, name='Qmax_s_')    # shape=(None, )
            self.q_target = tf.stop_gradient(q_target)
        with tf.variable_scope('q_eval'):
            a_indices = tf.stack([tf.range(tf.shape(self.a)[0], dtype=tf.int32), self.a], axis=1)
            self.q_eval_wrt_a = tf.gather_nd(params=self.q_eval, indices=a_indices)    # shape=(None, )
        with tf.variable_scope('loss'):
            self.loss = tf.reduce_mean(tf.squared_difference(self.q_target, self.q_eval_wrt_a, name='TD_error'))
        with tf.variable_scope('train'):
            self._train_op = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss)

    def store_transition(self, s, a, r, s_):
        if not hasattr(self, 'memory_counter'):
            self.memory_counter = 0
        transition = np.hstack((s, [a, r], s_))
        # replace the old memory with new memory
        index = self.memory_counter % self.memory_size
        self.memory[index, :] = transition
        self.memory_counter += 1

    def choose_action(self, observation):
        # to have batch dimension when feed into tf placeholder
        observation = observation[np.newaxis, :]

        if np.random.uniform() < self.epsilon:
            # forward feed the observation and get q value for every actions
            actions_value = self.sess.run(self.q_eval, feed_dict={self.s: observation})
            action = np.argmax(actions_value)
        else:
            action = np.random.randint(0, self.n_actions)
        return action

    def learn(self):
        # check to replace target parameters
        if self.learn_step_counter % self.replace_target_iter == 0:
            self.sess.run(self.target_replace_op)
            #print('\ntarget_params_replaced\n')

        # sample batch memory from all memory
        if self.memory_counter > self.memory_size:
            sample_index = np.random.choice(self.memory_size, size=self.batch_size)
        else:
            sample_index = np.random.choice(self.memory_counter, size=self.batch_size)
        batch_memory = self.memory[sample_index, :]

        _, cost = self.sess.run(
            [self._train_op, self.loss],
            feed_dict={
                self.s: batch_memory[:, :self.n_features],
                self.a: batch_memory[:, self.n_features],
                self.r: batch_memory[:, self.n_features + 1],
                self.s_: batch_memory[:, -self.n_features:],
            })

        self.cost_his.append(cost)

        # increasing epsilon
        self.epsilon = self.epsilon + self.epsilon_increment if self.epsilon < self.epsilon_max else self.epsilon_max
        self.learn_step_counter += 1

    def plot_cost(self):
        import matplotlib.pyplot as plt
        plt.plot(np.arange(len(self.cost_his)), self.cost_his)
        plt.ylabel('Cost')
        plt.xlabel('training steps')
        plt.show()

# if __name__ == '__main__':
#     DQN = DeepQNetwork(3,4, output_graph=True)

import heapq
import random

class BikeNet():
    def __init__(self, N, R, A, Q, repair, P, time_limit):
        self.N = N
        self.R = R
        self.A = A
        self.areas = list(range(A + 1))
        self.Q = Q
        self.repair = repair
        self.time_limit = time_limit
        #self.car = start_position
        self.car = 0

        self.reset()

    def reset(self):
        self.T = 0
        self.carrier_position = self.car
        self.place = list(range(self.A))
        self.time = sorted([random.expovariate(self.R[i][0]) for i in self.place], reverse=True)
        self.type = [-1] * self.A
        self.state = [int(self.N / self.A), 0] * self.A
        self.scheduler = [[self.time[x], self.type[x], self.place[x]] for x in range(self.A)]
        heapq.heapify(self.scheduler)
        return np.array(self.state.copy())

    def step(self, action):
        rewards = 0
        # time for carrier to take the action and repair one bicycle
        t = (abs(self.carrier_position % (self.A - 1) - action % (self.A - 1)) + abs(
            self.carrier_position // (self.A - 1) - action // (self.A - 1))) *0.5 + self.repair
        t_cursor = self.T + t
        event = self.scheduler[0]
        self.T, kind, location = event[0], event[1], event[2]
        self.carrier_position = action

        # update the atate of QN during the tansformation time
        while self.T < t_cursor:
            if kind == 1:  # 车到达
                self.state[location] += 1
            else:
                if self.state[location] == 0:  # 顾客到达，但没车
                    rewards -= 1
                else:
                    target = np.random.choice(self.areas, 1, p=self.Q[location])[0]
                    if target == self.A:  # 顾客到达，发现是坏车
                        self.state[location] -= 1
                        self.state[location + 1] += 1
                        continue
                    else:  # 顾客到达，顺利骑行
                        self.state[location] -= 1
                        next_time = random.expovariate(self.R[location][0]) + self.T
                        if next_time <= self.time_limit:
                            heapq.heappush(self.scheduler, [next_time, 1, target])
                next_time = random.expovariate(self.R[location][1]) + self.T
                if next_time <= self.time_limit:
                    heapq.heappush(self.scheduler, [next_time, -1, location])
            heapq.heappop(self.scheduler)
            if self.scheduler:
                event = self.scheduler[0]
                self.T, kind, location = event[0], event[1], event[2]
            else:
                break

        self.T = t_cursor
        if self.state[self.carrier_position+1] > 0:
            self.state[self.carrier_position] += 1
            self.state[self.carrier_position + 1] -= 1
        s_ = np.array(self.state.copy())

        if self.T <= self.time_limit and self.scheduler:
            return s_, rewards, 0
        else:
            return s_, rewards, 1
# from maze_env import Maze
# from RL_brain import DeepQNetwork
from tqdm import tqdm
from numba import jit


@jit
def simulate():
    #step = 0
    n_episodes = 500
    result = []
    for episode in tqdm(range(n_episodes)):
        sum_r = 0
        # initial observation
        observation = env.reset()
        action = 0
        #observation = np.array(int(N/A)) #devide all the normal bikes to all the areas evenly at the beginning

        while True:
            # fresh env
            #env.render()

            # RL choose action based on observation
            #if not env.a[action].broken_bike:
            action = RL.choose_action(observation)
            #    action = (action+1)%A

            # RL take action and get next observation and reward
            observation_, reward, done = env.step(action)

            #RL.store_transition(observation, action, reward, observation_)

#             if (step > 200) and (step % 5 == 0):
#                 RL.learn()
            #RL.learn()

            # swap observation
            observation = observation_

            # break while loop when end of this episode
            if done:
                break
            #step += 1
            sum_r += reward
            
        result.append([episode, sum_r])

    # end of game
    print('learning over')
    return result


if __name__ == "__main__":
    # maze game
    np.random.seed(1)
    N = 80 #total number of bikes in the QN
    A = 4 #A for areas, indicates the aumber of areas and the action space
    #R = pd.DataFrame({'cus_arr': [1] * A, 'ride': [0.5] * A}, index=range(A))
    R = {}  # [customer_arrval, ride]
    for i in range(A): R[i] = [1.0, 2.0]
    Q = [np.random.rand(A) for i in range(A)]
    Q = [q / sum(q)*0.99 for q in Q]
    Q = [np.append(q, 0.01) for q in Q]
    #Q = [[0,0.9,0.1], [0.9,0,0.1]]
    t_repair = 5
    P = 0
    time_limit = 1800

    env = BikeNet(N, R, A, Q, t_repair, P, time_limit)
    
    RL = DeepQNetwork(A, 2*A,
                      learning_rate=0.01,
                      reward_decay=0.9,
                      e_greedy=0.9,
                      replace_target_iter=200,
                      memory_size=2000,
                      # output_graph=True
                      )
    output = simulate()
    #RL.plot_cost()

100%|████████████████████████████████████████| 500/500 [05:49<00:00,  1.39it/s]


learning over


In [3]:
output = np.array(output)
output[:,1].mean()

-4516.054